In [22]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense
import os
import time as tm
from random import seed,randint

In [5]:
folder = 'D:\\Yolo\\facerec\\'

In [6]:
def click_images(name):
    
    face_detect = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')
    img_folder = os.path.join(folder,name)
    
    if not os.path.exists(img_folder):
        os.mkdir(img_folder)
        os.chdir(img_folder)
    else:
        print('Folder exists...')
        os.chdir(img_folder)
        
    st = 3
    while st>=1:
        print('Taking picture in...',st)
        st -= 1
        tm.sleep(1)
    
    cam = cv2.VideoCapture(0)
    pic = 0
    try:
        while pic<500:
            pic += 1
            _,img = cam.read()
            img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            face = face_detect.detectMultiScale(img,scaleFactor=1.1,minNeighbors=7,minSize=(40,40))
            for (x,y,w,h) in face:
                img = img[y:y+h,x:x+w]
            img = cv2.resize(img,(50,50))
            cv2.imwrite(os.path.join(img_folder,f'{pic}.jpg'),img)
            print(pic,':clicked!')
    except Exception as e:
        print("Error occured:",e)
        cam.release()
        exit(0)
    
    cam.release()

In [15]:
def getData(name):
    img_data = []
    fol_path = os.path.join(folder,name)
    for img in os.listdir(fol_path):
        imgarr = cv2.imread(img,cv2.IMREAD_GRAYSCALE)
        img_data.append([np.array(imgarr),1])
        # np.array([1,0])
        print(img,'->done')
    return img_data

In [8]:
name = input('Enter name')
click_images(name)

Taking picture in... 3
Taking picture in... 2
Taking picture in... 1
1 :clicked!
2 :clicked!
3 :clicked!
4 :clicked!
5 :clicked!
6 :clicked!
7 :clicked!
8 :clicked!
9 :clicked!
10 :clicked!
11 :clicked!
12 :clicked!
13 :clicked!
14 :clicked!
15 :clicked!
16 :clicked!
17 :clicked!
18 :clicked!
19 :clicked!
20 :clicked!
21 :clicked!
22 :clicked!
23 :clicked!
24 :clicked!
25 :clicked!
26 :clicked!
27 :clicked!
28 :clicked!
29 :clicked!
30 :clicked!
31 :clicked!
32 :clicked!
33 :clicked!
34 :clicked!
35 :clicked!
36 :clicked!
37 :clicked!
38 :clicked!
39 :clicked!
40 :clicked!
41 :clicked!
42 :clicked!
43 :clicked!
44 :clicked!
45 :clicked!
46 :clicked!
47 :clicked!
48 :clicked!
49 :clicked!
50 :clicked!
51 :clicked!
52 :clicked!
53 :clicked!
54 :clicked!
55 :clicked!
56 :clicked!
57 :clicked!
58 :clicked!
59 :clicked!
60 :clicked!
61 :clicked!
62 :clicked!
63 :clicked!
64 :clicked!
65 :clicked!
66 :clicked!
67 :clicked!
68 :clicked!
69 :clicked!
70 :clicked!
71 :clicked!
72 :clicked!
73 :

In [16]:
data = getData(name)
print(len(data))

1.jpg ->done
10.jpg ->done
100.jpg ->done
101.jpg ->done
102.jpg ->done
103.jpg ->done
104.jpg ->done
105.jpg ->done
106.jpg ->done
107.jpg ->done
108.jpg ->done
109.jpg ->done
11.jpg ->done
110.jpg ->done
111.jpg ->done
112.jpg ->done
113.jpg ->done
114.jpg ->done
115.jpg ->done
116.jpg ->done
117.jpg ->done
118.jpg ->done
119.jpg ->done
12.jpg ->done
120.jpg ->done
121.jpg ->done
122.jpg ->done
123.jpg ->done
124.jpg ->done
125.jpg ->done
126.jpg ->done
127.jpg ->done
128.jpg ->done
129.jpg ->done
13.jpg ->done
130.jpg ->done
131.jpg ->done
132.jpg ->done
133.jpg ->done
134.jpg ->done
135.jpg ->done
136.jpg ->done
137.jpg ->done
138.jpg ->done
139.jpg ->done
14.jpg ->done
140.jpg ->done
141.jpg ->done
142.jpg ->done
143.jpg ->done
144.jpg ->done
145.jpg ->done
146.jpg ->done
147.jpg ->done
148.jpg ->done
149.jpg ->done
15.jpg ->done
150.jpg ->done
151.jpg ->done
152.jpg ->done
153.jpg ->done
154.jpg ->done
155.jpg ->done
156.jpg ->done
157.jpg ->done
158.jpg ->done
159.jpg ->done
16.

In [17]:
train = data[:460]
test = data[460:]

X_train = np.array([i[0] for i in train]).reshape((-1,50,50,1))
y_train = np.array([i[1] for i in train])
X_test = np.array([i[0] for i in test]).reshape((-1,50,50,1))
y_test = np.array([i[1] for i in test])

print(X_train.shape)
print(X_test.shape)

(460, 50, 50, 1)
(40, 50, 50, 1)


In [18]:
model = Sequential()
model.add(Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),activation='relu',input_shape=(50,50,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=128,kernel_size=(3,3),strides=(1,1),activation='relu'))
model.add(Flatten())
model.add(Dense(units=128,activation='relu'))
model.add(Dense(units=1,activation='sigmoid'))
model.summary()
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 48, 48, 32)        320       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 24, 24, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 22, 22, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 11, 11, 64)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 9, 9, 128)         73856     
                                                                 
 flatten_2 (Flatten)         (None, 10368)            

In [21]:
results = model.fit(X_train,y_train,epochs=10,validation_data=(X_test,y_test))

Epoch 1/10
15/15 [==============================] - 4s 236ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/10
15/15 [==============================] - 3s 224ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/10
15/15 [==============================] - 4s 290ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/10
15/15 [==============================] - 3s 225ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/10
15/15 [==============================] - 3s 200ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/10
15/15 [==============================] - 3s 207ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/10
15/15 [==============================] - 3s 208ms/step - loss: 0.0000e+00 - accuracy: 1.0

In [60]:
folder_test = os.path.join(folder,'test_imgs','plant.jpg')
# for imgs in os.listdir(folder_test):
#     # print(imgs)
#     test_img = cv2.imread(imgs)
#     print(test_img)
#     test_img = cv2.cvtColor(test_img,cv2.COLOR_BGR2GRAY)
#     test_img = np.array([test_img]).reshape((-1,50,50,1))
    # predict = model.predict(test_img)
    # if predict[0][0]<0.5:
    #     print('No user...')
    # else:
    #     print('Present user')
img = cv2.imread(folder_test)
img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
img = cv2.resize(img,(50,50))
test_img = np.array([img]).reshape((-1,50,50,1))
cv2.imwrite(os.path.join(folder,'new.jpg'),img)
print(test_img.shape)
predict = model.predict(test_img)
print(predict)
if predict[0][0]<1:
    print('No user...')
else:
    print('Present user')

(1, 50, 50, 1)
1/1 [==============================] - 0s 216ms/step
[[1.]]
Present user
